# Visualize your mobility with Google Maps Timeline
You can view and manage your Location History information through <a href="https://www.google.com/maps/timeline">Google Maps Timeline</a>, which is available to both mobile and desktop users. <u>Your Timeline is private, so only you can see it</u>. Please download your Timeline data following the instructions a this <a href="https://support.google.com/accounts/answer/3024190">link</a>. 

If you have other settings like Web & App Activity turned on and you pause Location History or delete location data from Location History, you may still have location data saved in your Google Account as part of your use of other Google sites, apps, and services. For example, location data may be saved as part of activity on Search and Google Maps when your Web & App Activity setting is on, and included in your photos, depending on your camera app settings.

Note: if your notebook does not visualize widgets, you find possible solutions at this <a href="https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show">link</a>.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import skmob
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import folium
from folium.plugins import DualMap
from folium.map import Marker
from folium import LayerControl
import numpy as np
from skmob.io.file import load_google_timeline
from skmob.preprocessing import detection, clustering
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from datetime import datetime
from IPython.display import clear_output
from skmob.measures.individual import radius_of_gyration, maximum_distance, uncorrelated_entropy
from skmob.measures.individual import number_of_locations, waiting_times, real_entropy
from skmob.measures.individual import waiting_times, max_distance_from_home
from skmob.measures.collective import visits_per_time_unit
import warnings
warnings.filterwarnings('ignore')

In [2]:
filename = widgets.Text(
    value='Cronologia delle posizioni.json',
    placeholder='Type something',
    description='filename:',
    disabled=False,
    continuous_update=False,
)

region_text = widgets.Text(
    value='Tuscany, Italy',
    placeholder='Type something',
    description='Region:',
    disabled=False,
    continuous_update=False,
)

meters_slider = widgets.IntSlider(
    value=500,
    min=100,
    max=10000,
    step=100,
    description='Cell size (m):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    #layout={'width': '600px'},
)


checkbox1 = widgets.Checkbox(
    value=True,
    description='All trips',
    disabled=False,
    indent=False
)

checkbox2 = widgets.Checkbox(
    value=False,
    description='Trips in region',
    disabled=False,
    indent=False
)

button = widgets.Button(description="OK")

first_start, first_end = datetime(2019, 3, 1), datetime(2019, 4, 14)
second_start, second_end = datetime(2020, 3, 1), datetime(2019, 4, 14)
def load_trajs():
    
    tdf = load_google_timeline('data/' + filename.value).drop(['activity', 'altitude', 'heading', 
                                                                               'velocity', 'verticalAccuracy'], 
                                                                       axis=1).sort_values(by='datetime').drop('accuracy', axis=1)
    tdf['uid'] = 1
    
    # TrajDataFrame of first period
    tdf1 = tdf[(tdf.datetime > first_start) & (tdf.datetime < first_end)]
    # TrajDataFrame of second period
    tdf2 = tdf[(tdf.datetime > second_start)]
    if checkbox1.value:
        map_f = folium.plugins.DualMap(location=(tdf2.lat.mean(), tdf2.lng.mean()), 
                                       tiles='cartodbpositron', zoom_start=6)
        m1, m2 = map_f.m1, map_f.m2
        
        tdf1.plot_trajectory(map_f=m1, start_end_markers=False, hex_color='red', 
                    tiles='openstreetmap')
        tdf2.plot_trajectory(map_f=m2, start_end_markers=False, 
                                     hex_color='blue')
        
        display(map_f)
    
    region = region_text.value
    meters = 5000 #int(meters_slider.value)
    tessellation = tilers.tiler.get("squared", base_shape=region, meters=meters)
    
    # For the first TrajDataFrame
    ftdf1 = skmob.preprocessing.filtering.filter(tdf1)
    stdf1 = detection.stops(ftdf1, minutes_for_a_stop=10)
    mapped_stdf1 = stdf1.mapping(tessellation, remove_na=True)
    ctdf1 = clustering.cluster(mapped_stdf1, cluster_radius_km=0.1)
    distinct_locs1 = ctdf1.drop_duplicates(subset='cluster')
    
    # For the second TrajDataFrame
    ftdf2 = skmob.preprocessing.filtering.filter(tdf2)
    stdf2 = detection.stops(ftdf2, minutes_for_a_stop=10)
    mapped_stdf2 = stdf2.mapping(tessellation, remove_na=True)
    ctdf2 = clustering.cluster(mapped_stdf2, cluster_radius_km=0.1)
    distinct_locs2 = ctdf2.drop_duplicates(subset='cluster')
    
    if checkbox2.value:
        
        map_f = folium.plugins.DualMap(location=(mapped_stdf1.lat.mean(), mapped_stdf1.lng.mean()), 
                                       tiles='cartodbpositron', zoom_start=12)
        m1 = map_f.m1
        m2 = map_f.m2
        
        m1 = ctdf1.plot_trajectory(map_f=m1, start_end_markers=False, hex_color='red', 
                               opacity=0.5, weight=1)
        m1 = distinct_locs1.plot_stops(map_f=m1, radius=8)
        
        m2 = ctdf2.plot_trajectory(map_f=m2, start_end_markers=False, hex_color='blue', 
                                       opacity=0.5, weight=1)
        m2 = distinct_locs2.plot_stops(map_f=m2, radius=8)
    
        display(map_f)

    fig, ax = plt.subplots(1, 3, figsize=(15, 6))
        
    nlocs1 = number_of_locations(mapped_stdf1, show_progress=False, ).iloc[0][1]
    nlocs2 = number_of_locations(mapped_stdf2, show_progress=False, ).iloc[0][1]
    ax[0].bar([1], [len(ctdf1.cluster.unique())], color='red', alpha=0.75)
    ax[0].bar([2], [len(ctdf2.cluster.unique())], color='blue', alpha=0.75)
    ax[0].set_ylabel('locations visited', fontsize=15)
    ax[0].set_xticks([1, 2])
    ax[0].set_xticklabels(['2019', '2020'], rotation=45, fontsize=15) 
    ax[0].grid(alpha=0.2)
    
    rg1 = radius_of_gyration(tdf1, show_progress=False).iloc[0][1]
    rg2 = radius_of_gyration(tdf2, show_progress=False).iloc[0][1]
    ax[1].bar([1], [rg1], color='red', alpha=0.75)
    ax[1].bar([2], [rg2], color='blue', alpha=0.75)
    ax[1].set_ylabel('mobility volume [km]', fontsize=15)
    ax[1].set_xticks([1, 2])
    ax[1].set_xticklabels(['2019', '2020'], rotation=45, fontsize=15)
    ax[1].grid(alpha=0.2)
    
    max_d1 = maximum_distance(ftdf1, show_progress=False).iloc[0][1]
    max_d2 = maximum_distance(ftdf2, show_progress=False).iloc[0][1]
    ax[2].bar([1], [max_d1], color='red', alpha=0.75)
    ax[2].bar([2], [max_d2], color='blue', alpha=0.75)
    ax[2].set_ylabel('maximum distance', fontsize=15)
    ax[2].set_xticks([1, 2])
    ax[2].set_xticklabels(['2019', '2020'], rotation=45, fontsize=15)
    ax[2].grid(alpha=0.2)
    
    plt.subplots_adjust(wspace=0.4)
    plt.show()

def on_button_clicked(b):
    clear_output(wait=True)
    display(vbox) 
    load_trajs()
    
button.on_click(on_button_clicked)

vbox = widgets.VBox([filename, region_text, 
                     checkbox1, checkbox2, button])

In [3]:
display(vbox)